<h1><b>Import Libararies


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

<h1><b> Define Functions

In [ ]:
def preprocess_func(input):
       input['medal type']=input['medal_type'].str.lower()
       input['country']=input['country_name']+'_'+input['medal type']
       input['slug_game'] = input['slug_game'].str.extract('(\d+)').astype(int)
       input.drop(['athlete_url'	,'athlete_full_name'	,'country_name'	,'country_code',
          'participant_title'	,'country_3_letter_code'	,'medal type'],axis=1,inplace=True)

<h1><b>Read Data

In [ ]:
ss=pd.read_csv('SampleSubmission.csv')
data=pd.read_csv('data.csv')

<h1><b> Preprocessing

In [ ]:
preprocess_func(data)

In [ ]:
data['type']=data['participant_type'].replace(['GameTeam', 'Athlete'],[1,2])
data['medals']=data['medal_type'].replace(['GOLD','SILVER','BRONZE'],[1,1,1])
data['gender']=data['event_gender'].replace(['Mixed', 'Women', 'Men', 'Open'],[1,2,3,4])
data['medal_type']=data['medal_type'].replace(['GOLD','SILVER','BRONZE'],[1,2,3])


In [ ]:
data.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
feats=['event_title','discipline_title']
le=LabelEncoder()
df=data
for f in feats :
    le.fit(df[f])
    df[f]=le.transform(df[f])
data.drop(['participant_type','event_gender'],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
grouped = data.groupby(['country','slug_game'])
result = grouped.agg({
    'medals': 'sum',
    'type':'sum',
    'event_title':'median' ,
    'discipline_title':'median' ,
    'gender':'median'
})


In [ ]:
result= result.reset_index().rename(columns={'index': 'country'})
result.sort_values(['slug_game'], ignore_index=True, inplace=True)

In [ ]:
train=result[:3460]
test=result[3460:]
test_df=ss.copy()

In [ ]:
test_df.drop('Target',axis=1,inplace=True)

In [ ]:
# train.hist()

In [ ]:
test.head()

In [ ]:
for i in range(test_df.shape[0]):
  country = test_df.iloc[i]['Country']
  if country in test['country'].values:
    test_df.loc[i, 'gender'] = test.loc[test['country'] == country, 'gender'].values[0]
    test_df.loc[i, 'type'] = test.loc[test['country'] == country, 'type'].values[0]
    test_df.loc[i, 'discipline_title'] = test.loc[test['country'] == country, 'discipline_title'].values[0]
    test_df.loc[i, 'slug_game'] = test.loc[test['country'] == country, 'slug_game'].values[0]
    test_df.loc[i, 'event_title'] = test.loc[test['country'] == country, 'event_title'].values[0]
train.drop('country',axis=1,inplace=True)


In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, KFold,StratifiedKFold,GroupKFold,TimeSeriesSplit
from sklearn.ensemble import StackingRegressor,VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
X=train.drop('medals',axis=1)
y=train['medals'].copy()
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=10)

In [ ]:
estimators=[
    # ('lr',KNeighborsRegressor()),
        ('rd',Ridge()),
        ('etr',ExtraTreesRegressor()),
     ('gb',GradientBoostingRegressor()),
                                ('xgb',XGBRegressor()),
                               ('lgbm',LGBMRegressor()),
                               ('rf',RandomForestRegressor()),
                               ('hgb',HistGradientBoostingRegressor()),
                               ('cat',CatBoostRegressor()),]
xg=VotingRegressor(estimators=estimators)
xg.fit(X_train,y_train)
y_pred=xg.predict(X_test)
np.sqrt(mean_squared_error(y_pred,y_test))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000280 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 495
[LightGBM] [Info] Number of data points in the train set: 2768, number of used features: 5
[LightGBM] [Info] Start training from score 6.321171
Learning rate set to 0.048089
0:	learn: 8.7895046	total: 1.34ms	remaining: 1.33s
1:	learn: 8.4213430	total: 4.13ms	remaining: 2.06s
2:	learn: 8.0876605	total: 7.77ms	remaining: 2.58s
3:	learn: 7.7741334	total: 11.5ms	remaining: 2.87s
4:	learn: 7.4813230	total: 15ms	remaining: 2.99s
5:	learn: 7.1783242	total: 18.5ms	remaining: 3.06s
6:	learn: 6.9093978	total: 22ms	remaining: 3.12s
7:	learn: 6.6406867	total: 26.9ms	remaining: 3.33s
8:	learn: 6.3822907	total: 29ms	remaining: 3.19s
9:	learn: 6.1371088	total: 33.2ms	remaining: 3.28s
10:	learn: 5.8942302	total: 36.2ms	remaining: 3.26s
11:	learn: 5.6656010	total: 39.8ms	remaining: 3.28s
12:	learn: 5.4592020	to

0.706527078209703

In [ ]:
test_df=test_df.fillna(0)

In [ ]:
ID=test_df['Country']

In [ ]:
test_df.drop('Country',axis=1,inplace=True)

In [ ]:
test_df = test_df[X.columns]

In [ ]:
xg.fit(X,y)
test_df['Target']=xg.predict(test_df)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000095 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 508
[LightGBM] [Info] Number of data points in the train set: 3460, number of used features: 5
[LightGBM] [Info] Start training from score 6.168208
Learning rate set to 0.049814
0:	learn: 8.4666997	total: 1.45ms	remaining: 1.45s
1:	learn: 8.0966507	total: 3.14ms	remaining: 1.57s
2:	learn: 7.7630988	total: 4.68ms	remaining: 1.55s
3:	learn: 7.4439423	total: 6.16ms	remaining: 1.53s
4:	learn: 7.1501650	total: 7.65ms	remaining: 1.52s
5:	learn: 6.8467529	total: 9.07ms	remaining: 1.5s
6:	learn: 6.5813266	total: 10.6ms	remaining: 1.5s
7:	learn: 6.3278785	total: 12ms	remaining: 1.49s
8:	learn: 6.0588385	total: 13.5ms	remaining: 1.48s
9:	learn: 5.8112520	total: 14.9ms	remaining: 1.48s
10:	learn: 5.5703721	total: 16.4ms	remaining: 1.47s
11:	learn

In [ ]:
ss['Target'] = ss['Target'].apply(lambda x: 0 if x < 1 else round(x))

In [ ]:
ss.to_csv("submission.csv",index=False)

In [ ]:
ss.head()

,Country,Target
0,Algeria_gold,0
1,Angola_gold,0
2,Benin_gold,0
3,Botswana_gold,0
4,Burkina Faso_gold,0
